# Chinook Database Exploration with SQLAlchemy & pandas

This notebook walks through the exercises using **SQLAlchemy** (Core + ORM) and **pandas** against the classic **Chinook** SQLite database.

> **How to run:**  
> 1) Ensure you have **sqlalchemy**, **pandas**, and **ipython** installed.  
> 2) Place `chinook.db` in the **same folder** as this notebook (or edit `DB_PATH` below).  
> 3) Run cells top-to-bottom.

We will:
- Connect to the database using SQLAlchemy
- Reflect the existing schema
- Execute basic SQL queries
- Use the ORM for joins
- Fetch results into pandas for display/analysis


In [ ]:
# 🔧 Setup
# If needed, install packages in your environment (uncomment if running locally)
# !pip install sqlalchemy pandas ipython

import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

print('pandas:', pd.__version__)
print('sqlalchemy:', sqlalchemy.__version__)

In [ ]:
# 🔗 Paths
DB_PATH = 'chinook.db'  # change this if your DB is elsewhere

assert os.path.exists(DB_PATH), f'Could not find {DB_PATH}. Place the file next to this notebook or update DB_PATH.'

## Helper functions (display SQL/pandas results neatly)

In [ ]:
from IPython.display import display

def sql(query):
    print()
    print(query)
    print()

def get_results(query_or_statement, engine):
    """Accepts an ORM Query, a SQLAlchemy Core statement, or a raw SQL string."""
    if hasattr(query_or_statement, 'statement'):  # ORM Query
        q = query_or_statement.statement
    else:
        q = query_or_statement
    return pd.read_sql(q, engine)

def display_results(query_or_statement, engine):
    df = get_results(query_or_statement, engine)
    display(df)
    sql(query_or_statement)

## Exercise 1 — Open the database & reflect schema

In [ ]:
# Create the engine and test a connection
engine = create_engine(f'sqlite:///{DB_PATH}', future=True)
with engine.connect() as conn:
    print('Connected OK:', conn)

# Reflect schema metadata
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

# Automap ORM classes
Base = automap_base(metadata=metadata)
Base.prepare()

# Create a Session
Session = sessionmaker(bind=engine, future=True)
session = Session()

# Small helper to list mapped classes
def mapped_classes_dict():
    out = {}
    for name, obj in Base.classes.__dict__.items():
        if not name.startswith('_') and hasattr(obj, '__table__'):
            out[obj.__table__.name] = obj
    return out

mapped = mapped_classes_dict()
print('Mapped tables -> ORM classes:')
for t, cls in sorted(mapped.items()):
    print(f'  {t:15s} -> {cls.__name__}')

## Exercise 2 — Table names

In [ ]:
inspector = inspect(engine)
tables = inspector.get_table_names()
print('Tables:', tables)

## Exercise 3 — Tracks (first three rows)

In [ ]:
Tracks = mapped['tracks']  # ORM class for the tracks table
q = session.query(Tracks).limit(3)
display_results(q, engine)

## Exercise 4 — Track name and album title for first 20 tracks

In [ ]:
Albums = mapped['albums']
q = (session.query(Tracks.Name.label('TrackName'), Albums.Title.label('AlbumTitle'))
     .join(Albums, Tracks.AlbumId == Albums.AlbumId)
     .order_by(Tracks.TrackId.asc())
     .limit(20))
display_results(q, engine)

## Exercise 5 — First 10 track sales from invoice_items (with track names & quantities)

In [ ]:
InvoiceItems = mapped['invoice_items']

q = (session.query(InvoiceItems.InvoiceLineId,
                   InvoiceItems.InvoiceId,
                   Tracks.Name.label('TrackName'),
                   InvoiceItems.Quantity,
                   InvoiceItems.UnitPrice)
     .join(Tracks, Tracks.TrackId == InvoiceItems.TrackId)
     .order_by(InvoiceItems.InvoiceLineId.asc())
     .limit(10))
display_results(q, engine)

## Exercise 6 — Top 10 tracks sold (by total quantity)

In [ ]:
q = (session.query(Tracks.Name.label('TrackName'),
                   func.sum(InvoiceItems.Quantity).label('TotalSold'))
     .join(InvoiceItems, Tracks.TrackId == InvoiceItems.TrackId)
     .group_by(Tracks.TrackId, Tracks.Name)
     .order_by(func.sum(InvoiceItems.Quantity).desc())
     .limit(10))
display_results(q, engine)

## Exercise 7 — Top 10 highest-selling artists (by total track quantities)

In [ ]:
Artists = mapped['artists']
Albums = mapped['albums']  # ensure defined

q = (session.query(Artists.Name.label('ArtistName'),
                   func.sum(InvoiceItems.Quantity).label('TotalSold'))
     .join(Albums, Artists.ArtistId == Albums.ArtistId)
     .join(Tracks, Tracks.AlbumId == Albums.AlbumId)
     .join(InvoiceItems, InvoiceItems.TrackId == Tracks.TrackId)
     .group_by(Artists.ArtistId, Artists.Name)
     .order_by(func.sum(InvoiceItems.Quantity).desc())
     .limit(10))
display_results(q, engine)

---
### Bonus: Using pandas-only SQL (optional)
You can also use `pd.read_sql` with raw SQL strings. For example:

```python
pd.read_sql("""
SELECT t.Name AS TrackName, SUM(ii.Quantity) AS TotalSold
FROM invoice_items AS ii
JOIN tracks AS t ON t.TrackId = ii.TrackId
GROUP BY t.TrackId, t.Name
ORDER BY TotalSold DESC
LIMIT 10;
""", engine)
```
